# Wrangle OpenStreetMap Data - Tri-Cities, Washington

## Introduction

> This purpose of this project is to take a certain area of land and wrangle the data. Extract and clean the data to make it much more uniform and readable. The area I have chosen is an area called 'Tri-Cities' in Washington State. I chose this area as I have lived here for a long time. The original OpenStreetMap data I am using for this project can be found and exported [Here](https://www.openstreetmap.org/export#map=11/46.2421/-119.1886). I used an API called Overpass API to export this data. The documentation for Overpass API can be found [Here](https://wiki.openstreetmap.org/wiki/Overpass_API). The finished project and all files will be uploaded to [This GitHub Repository](https://github.com/TrikerDev/Wrangle-OpenStreetMap-Data---Tri-Cities---Washington).

# Data Gathering

## Importing

In [1]:
# Importing OSM File and packages

import xml.etree.ElementTree as ET
import pprint
from collections import defaultdict
import pandas
import re
import csv
import codecs
import sqlite3
import cerberus
import schema
import os


OSM_FILE = "map.osm"

## Gathering Tags

In [2]:
# Getting tags
def get_element(filename, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(filename, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

## Fetching Tags

In [3]:
# Counting and returning tag counts per type of tag
def count_tags(filename):
    tree=ET.iterparse(filename)
    tags={}
    for event,elem in tree:
        if elem.tag not in tags.keys():
            tags[elem.tag]=1
        else:
            tags[elem.tag] = tags[elem.tag]+1
    return tags    
    
with open(OSM_FILE,'rb') as f:
    tags=count_tags(OSM_FILE)
    pprint.pprint(tags)
f.close()

{'bounds': 1,
 'member': 15770,
 'meta': 1,
 'nd': 434301,
 'node': 368263,
 'note': 1,
 'osm': 1,
 'relation': 1431,
 'tag': 158810,
 'way': 38417}


## Finding the 'K' value of each tag and adding to a dictionary

>*  "lower", for tags that contain only lowercase letters and are valid
>*  "lower_colon", for otherwise valid tags with a colon in their names
>*  "problemchars", for tags with problematic characters
>*  "other", for other tags that do not fall into the other three categories

In [4]:
# Adding variables for the tags to be stored under, definitions above
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
    
# Function starting on our dataset
def process_keys_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

# Iterating through tags and adding them up
def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] = keys['problemchars'] + 1
        else:    
            keys['other'] += 1  
    return keys


# Opening file and starting process_keys_map function
with open(OSM_FILE,'rb') as f:
    keys = process_keys_map(OSM_FILE)
    pprint.pprint(keys)

# Closing file
f.close()

# Below is the count of each type of variable

{'lower': 111439, 'lower_colon': 43264, 'other': 4106, 'problemchars': 1}


## Getting number of unique users that contributed to the map data

In [5]:
# Function to count users
def count_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.get('user'):
            users.add(get_user(element))
        element.clear()    
    return users

# Fetching users function
def get_user(element):
    return element.get('user')

# Opening file and starting count_users function
with open(OSM_FILE,'rb') as f:
    users = count_users(OSM_FILE)

print (len(users))

f.close()

# user count below

453


In [6]:
# Printing all unique usernames
pprint.pprint(users)

set(['25or6to4',
     '503Greg',
     'AE35',
     'Aaron Lidman',
     'AaronAsAChimp',
     'Acefirst',
     'Adam Brault',
     'Adam Schneider',
     'Adamant1',
     'Akgeo',
     'Aleksandar Matejevic',
     'Alex-a',
     'AmandaCora',
     'Amnesiac9',
     'Amoebabadass',
     'AndjelaS',
     'Andre Engels',
     'Andre68',
     'AndrewSnow',
     'AntonioPigafetta',
     'ArizonaMapper',
     'ArminGh',
     'Audiophase',
     'AzrielB',
     'Azuka',
     'BCNorwich',
     'BadRegEx',
     'Baloo Uriza',
     'BaseballNut51',
     'Belobog',
     'Benny Goodman',
     'Black Cardinal',
     'Brad Meteor',
     'Brian Bradford',
     'Brian Reavis',
     'Brian@Brea',
     'Bryce C Nesbitt',
     'CarniLvr79',
     'Carnildo',
     'Chenshi',
     'Chetan_Gowda',
     'ChrisZontine',
     'Christopher-0118',
     'Chroma187',
     'Claudius Henrichs',
     'DJ Cane',
     'DKDestroyer',
     'Daniel C Berman',
     'DannyAiquipa',
     'DanteVento',
     'DareDJ',
     'Darr

## Getting Street Names

In [7]:
def street_names(filename):
        key='addr:street'
        values=[]
        EL=get_element(filename, tags=('node', 'way', 'relation'))
        for element in EL:
            for tag in element.iter('tag'):
                if tag.attrib['k']==key:
                    values.append(tag.attrib['v'])
            element.clear()
        print (key)
        pprint.pprint(values)
street_names(OSM_FILE)

addr:street
['West 27th Avenue',
 'Hunt Avenue',
 'Fowler Street',
 'Clearwater Avenue',
 'Tapteal Drive',
 'Willamette Ave',
 'Queensgate Drive',
 'Lee Boulevard',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Lee Boulevard',
 'Williams Boulevard',
 'Williams Boulevard',
 'Queensgate Drive',
 'Jadwin Avenue',
 'Jadwin Avenue',
 'Lindberg Loop',
 'Serivce Road',
 'Bronco Lane',
 'Bronco Lane',
 'Bronco Lane',
 'Butler Loop',
 'Butler Loop',
 'Airport Way',
 'Airport Way',
 'Aviator Drive',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Terminal Drive',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Airport Way',
 'Airport Way',
 'Butler Loop',
 'Butler Loop',
 'Butler Loop',
 'Lindberg Loop',
 'Terminal Drive',
 'Terminal Drive',
 'Terminal Drive',
 'Terminal Drive',
 'Terminal Drive',
 'Ter

> The main issue here is that the street name endings are not uniform. For example, some streets end in 'Drive' while others end in 'Dr', or 'Avenue' and 'Ave'. We want all the street name endings to be uniform.

## Getting Post Codes

In [8]:
def post_codes(filename):
        key='addr:postcode'
        values=[]
        EL=get_element(filename, tags=('node', 'way', 'relation'))
        for element in EL:
            for tag in element.iter('tag'):
                if tag.attrib['k']==key:
                    values.append(tag.attrib['v'])
            element.clear()
        print (key)
        pprint.pprint(values)
post_codes(OSM_FILE)

addr:postcode
['99337',
 '99354',
 '99352',
 '99352',
 '99352',
 '99352',
 '99352',
 '99352',
 '99352',
 '99352',
 '99352',
 '99354',
 '99354',
 '99352',
 '99352',
 '99352',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99354',
 '99337',
 '99301',
 '99352',
 '99301',
 '99301',
 '99336',
 '99354',
 '99338-7319',
 '99338-7319',
 '99338-7319',
 '99338-7319',
 '99338-7319',
 '99338-7319',
 '99354',
 '99354',
 '99354',
 '99337',
 '99323',
 '99336',
 '99336',
 '99353',
 '98352',
 '99353',
 '99353',
 '99353',
 '99301',
 '99301',
 '99352',
 '99352',
 '99352',
 '99362',
 '99336',
 '99336',
 '99336',
 '99352',
 '99301',
 '9936',
 '83853

> The postcodes are actually relatively clean. The only issue is that the majority of the post codes are 5 digits, like '11111' where as a few of them are 9 digits, like '11111-1111'.

# Data Cleaning

## Auditing Street Names

> Setting values we expect and mappings to them from common endings

In [9]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# The values we expect to see. This is the end result of all street name endings we want
expected = ["Street", "Avenue","Loop", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Freeway","Circle","Strand","Sterling","Way","Highway",
            "Terrace","South","East","West","North","Landing"]

# Mapping shortened version of common street name endings to the ending we want
mapping = {
            " St ": " Street ",
            " St": " Street ",
            " St.": " Street ",
            " ST": " Street ",
            " Rd.": " Road ",
            " Rd ": " Road ",
            " Rd": " Road ",
            " Ave ": " Avenue ", 
            " Ave": " Avenue ", 
            " Ave.": " Avenue ",
            " Av ": " Avenue ", 
            " Dr ": " Drive ",
            " Dr.": " Drive",
            " Dr": " Drive",
            " Pl ": " Place",
            " Pl": " Place",
            " Blvd ": " Boulevard ",
            " Blvd": " Boulevard",
            " Blvd.": " Boulevard",
            " Ct ": " Court ",
            " Ct": " Court ",
            " Ctr": " Center",
            " Pl ": " Place ",
            " Ln ": " Lane ",
            " Cir ": " Circle ",
            " Wy": " Way ",
            " S ": " South ",
            " E ": " East ",
            " W ": " West ",
            " N ": "North"
}

> Adding expected street names to groups and adding unexpected ones to be handled later

In [10]:
# Handling expected and unexpected street names
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

> Sending street names to be audited

In [11]:
# Getting street names and sending them to audit_street_type function
def audit(file):
    file = open(file, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    file.close()
    return street_types

> Updating the ending names of streets to our preferred type

In [12]:
# Function for replace 'old' names with 'new' names
def update_street_name(name, mapping):
    for key,value in mapping.items():
        if key in name:
            return name.replace(key,value)
    return name 

def audit_street_name_tag(element): 
    street_name=element.get('v')
    m = street_type_re.search(street_name)
    if m:
        better_street_name=update_street_name(street_name,mapping)
        return better_street_name
    return street_name

> Starting the street name auditing process

In [13]:
# Variable for new street names to be stored
st_types = audit(OSM_FILE)

for st_type, ways in st_types.items():
    for name in ways:
        better_name = update_street_name(name, mapping)
        print (name, "=>", better_name)

('So. Kent St.', '=>', 'So. Kent Street .')
('Road 36', '=>', 'Road 36')
('South Washington St', '=>', 'South Washington Street ')
('Cottonwood Creek Blvd', '=>', 'Cottonwood Creek Boulevard')
('S OCTAVE ST', '=>', 'S OCTAVE Street ')
('E. SR 397', '=>', 'E. SR 397')
('Road 72', '=>', 'Road 72')
('North Road 68', '=>', 'North Road 68')
('Willamette Ave', '=>', 'Willamette Avenue ')
('West Kennewick Ave', '=>', 'West Kennewick Avenue ')
('North Road 92', '=>', 'North Road 92')
('Indian Ridge Dr', '=>', 'Indian Ridge Drive')
('Tamarisk Dr', '=>', 'Tamarisk Drive')
('North Road 44', '=>', 'North Road 44')
('N Irving Pl', '=>', 'N Irving Place')
('Travis Ct', '=>', 'Travis Court ')


> Now all the common street endings have been updated to the full endings we want. However, there are some areas that may seem like a problem here. There are several endings with a number, such as 'Road 36'. This is actually okay and a correct name. There are many roads around this area that are numbered such as these. We will ignore changing the numbers of these roads because they are already correct.

## Auditing Post Codes

In [14]:
zip_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

zip_types = defaultdict(set)

expected_zip = {}

def audit_zip_codes(zip_types, zip_name, regex, expected_zip):
    m = regex.search(zip_name)
    if m:
        zip_type = m.group()
        if zip_type not in expected_zip:
             zip_types[zip_type].add(zip_name)

def is_zip_name(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit_zip(filename, regex):
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag == "way" or elem.tag == "node":
            for tag in elem.iter("tag"):
                if is_zip_name(tag):
                    audit_zip_codes(zip_types, tag.attrib['v'], regex, expected_zip)
    pprint.pprint(dict(zip_types)) 

    
audit_zip(OSM_FILE, zip_type_re)


for zip_type, ways in zip_types.iteritems(): 
        for name in ways:
            if "-" in name:
                name = name.split("-")[0].strip()
            elif len(str(name))>5:
                name=name[0:5]
                # This function because one of the postcodes in the dataset has only 4 digits. This is basically setting
                # it to 'unknown' as its all zeros
            elif len(str(name))<5:
                name = '00000'
            print name

{'83853': set(['83853']),
 '98352': set(['98352']),
 '99301': set(['99301']),
 '99320': set(['99320']),
 '99323': set(['99323']),
 '99336': set(['99336']),
 '99336-1117': set(['99336-1117']),
 '99337': set(['99337']),
 '99338': set(['99338']),
 '99338-7319': set(['99338-7319']),
 '99352': set(['99352']),
 '99353': set(['99353']),
 '99354': set(['99354']),
 '99354-2303': set(['99354-2303']),
 '9936': set(['9936']),
 '99362': set(['99362'])}
99323
99320
99337
99336
99301
99336
99338
98352
99338
99353
99352
83853
99354
99362
00000
99354


In [15]:
# The way to audit the postcodes is to make them all uniform. Since the vast majority of them are 5 digits, we will be
# auditing the longer codes by stripping off the extra digits and he '-'. This will make every post code a uniform 5 digits
def update_postcode(name): 
    if "-" in name:
        name = name.split("-")[0].strip()
    elif len(str(name))>5:
        name=name[0:5]
    elif len(str(name))<5:
        name = '00000'
    return name



def audit_postcode_tag(element,regex=re.compile(r'\b\S+\.?$', re.IGNORECASE)):
    post_code=element.get('v')
    m = regex.search(post_code)
    if m:
        better_postcode=update_postcode(post_code)
        return better_postcode
    return post_code

# Changing to CSV and Importing to a Database

## Preparing for Database

In [16]:
# Defining schema for SQL Database
schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

## Defining CSV files and fields

In [17]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


SCHEMA = schema


NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

## Shaping elements

In [18]:
# Cleaning and shaping elemnents to Python dict

"""Clean and shape node or way XML element to Python dict"""

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    
    
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = [] # Handle secondary tags the same way for both node and way elements
             

   

    if element.tag=='node':
        for field in node_attr_fields:
            node_attribs[field]=element.get(field)
                 
        if element.find('tag') is None:
            pass
           
        elif element.find('tag') is not None:
            tag_attrib={}
            node_tag_fields=NODE_TAGS_FIELDS
            for tag in element.iter('tag'):
                if PROBLEMCHARS.search(tag.attrib['k']):
                    pass
                elif LOWER_COLON.search(tag.attrib['k']):
                    tag_attrib[node_tag_fields[0]]=element.get('id')
                    tag_attrib[node_tag_fields[1]]=tag.get('k')[(tag.get('k').find(':')+1):]
                    if tag.attrib['k']== "addr:street":
                        tag_attrib[node_tag_fields[2]]=audit_street_name_tag(tag)
                    elif tag.attrib['k']== "addr:postcode":
                        tag_attrib[node_tag_fields[2]]=audit_postcode_tag(tag)       
                    else:
                        tag_attrib[node_tag_fields[2]]=tag.get('v')
                    tag_attrib[node_tag_fields[3]]=tag.get('k').split(':')[0]
                    tags.append(tag_attrib.copy())
                
                else:
                    tag_attrib[node_tag_fields[0]]=element.get('id')
                    tag_attrib[node_tag_fields[1]]=tag.get('k')
                    if tag.attrib['k']== "addr:street":
                        tag_attrib[node_tag_fields[2]]=audit_street_name_tag(tag)
                    elif tag.attrib['k']== "addr:postcode":
                        tag_attrib[node_tag_fields[2]]=audit_postcode_tag(tag)    
                    else:    
                        tag_attrib[node_tag_fields[2]]=tag.get('v')
                    tag_attrib[node_tag_fields[3]]=default_tag_type
                    tags.append(tag_attrib.copy())
             
        
                
    elif element.tag=='way':
        for field in way_attr_fields:
            way_attribs[field]=element.get(field)
    
        way_node_attrib={}
        way_node_fields=WAY_NODES_FIELDS
        for nd in element.findall('nd'):
            way_node_attrib[way_node_fields[0]]=element.get('id')
            way_node_attrib[way_node_fields[1]]=nd.get('ref')
            way_node_attrib[way_node_fields[2]]=element.findall('nd').index(nd)
            way_nodes.append(way_node_attrib.copy())
        
        
        
        
        if element.find('tag') is None:
            pass
           
        elif element.find('tag') is not None:
            way_tag_attrib={}
            way_tag_fields=WAY_TAGS_FIELDS
            for tag in element.iter('tag'):
                if PROBLEMCHARS.search(tag.attrib['k']):
                    pass
                elif LOWER_COLON.search(tag.attrib['k']):
                    way_tag_attrib[way_tag_fields[0]]=element.get('id')
                    way_tag_attrib[way_tag_fields[1]]=tag.get('k')[(tag.get('k').find(':')+1):]
                    if tag.attrib['k']== "addr:street":
                        way_tag_attrib[way_tag_fields[2]]=audit_street_name_tag(tag)
                    elif tag.attrib['k']== "addr:postcode":
                        way_tag_attrib[way_tag_fields[2]]=audit_postcode_tag(tag)    
                    else:
                        way_tag_attrib[way_tag_fields[2]]=tag.get('v')
                    way_tag_attrib[way_tag_fields[3]]=tag.get('k').split(':')[0]
                    tags.append(way_tag_attrib.copy())
                    
                else:
                    way_tag_attrib[way_tag_fields[0]]=element.get('id')
                    way_tag_attrib[way_tag_fields[1]]=tag.get('k')
                    if tag.attrib['k']== "addr:street":
                        way_tag_attrib[way_tag_fields[2]]=audit_street_name_tag(tag) 
                    elif tag.attrib['k']== "addr:postcode":
                        way_tag_attrib[way_tag_fields[2]]=audit_postcode_tag(tag)    
                    else:   
                        way_tag_attrib[way_tag_fields[2]]=tag.get('v')
                    way_tag_attrib[way_tag_fields[3]]=default_tag_type
                    tags.append(way_tag_attrib.copy())
        
    

    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

## Helper Functions

In [19]:
# Get Element
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [20]:
# Validate Element
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

In [21]:
# Dict Writer
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

## Main Function

In [22]:
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [23]:
# Starting the process
if __name__ == '__main__':
    process_map(OSM_FILE, validate=True)


# Creating the Database

In [24]:
# Connect to the database
mydb = 'tricities.db'
conn = sqlite3.connect(mydb)
cur = conn.cursor()

## Nodes Table

In [25]:
# Drop table if already exists
query="DROP TABLE IF EXISTS nodes;"
cur.execute(query);
conn.commit()
query = "CREATE TABLE nodes (id INTEGER PRIMARY KEY NOT NULL,lat REAL,lon REAL,user TEXT,uid INTEGER,version INTEGER,changeset INTEGER,timestamp TEXT);"
cur.execute(query)
conn.commit()

# Open CSV file and formatting data
with open('nodes.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['lat'].decode("utf-8"),i['lon'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

# Insert Data
cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)
conn.commit()
f.close()

## Nodes_Tags Table

In [26]:
# Drop table if already exists
query="DROP TABLE IF EXISTS nodes_tags;"
cur.execute(query);
conn.commit()
query = "CREATE TABLE nodes_tags (id INTEGER,key TEXT,value TEXT,type TEXT,FOREIGN KEY (id) REFERENCES nodes(id));"
cur.execute(query)
conn.commit()

# Open CSV file and formatting data
with open('nodes_tags.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

# Insert Data
cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
conn.commit()
f.close()

## Ways Table

In [27]:
# Drop table if already exists
query="DROP TABLE IF EXISTS ways;"
cur.execute(query);
conn.commit()
query = "CREATE TABLE ways(id INTEGER PRIMARY KEY NOT NULL,user TEXT,uid INTEGER,version TEXT,changeset INTEGER,timestamp TEXT);"
cur.execute(query)
conn.commit()

# Open CSV file and formatting data
with open('ways.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]
    
# Insert Data
cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_db)
conn.commit()
f.close()

## Ways_Tags Table

In [28]:
# Drop table if already exists
query="DROP TABLE IF EXISTS ways_tags;"
cur.execute(query);
conn.commit()
query = "CREATE TABLE ways_tags (id INTEGER NOT NULL,key TEXT NOT NULL,value TEXT NOT NULL,type TEXT,FOREIGN KEY (id) REFERENCES ways(id));"
cur.execute(query)
conn.commit()

# Open CSV file and formatting data
with open('ways_tags.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

# Insert Data
cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
conn.commit()
f.close()

## Ways_Nodes Table

In [29]:
# Drop table if already exists
query="DROP TABLE IF EXISTS ways_nodes;"
cur.execute(query);
conn.commit()
query = "CREATE TABLE ways_nodes (id INTEGER NOT NULL,node_id INTEGER NOT NULL,position INTEGER NOT NULL,FOREIGN KEY (id) REFERENCES ways(id),FOREIGN KEY (node_id) REFERENCES nodes(id));"
cur.execute(query)
conn.commit()

# Open CSV file and formatting data
with open('ways_nodes.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in dr]

# Insert Data
cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?,?,?);", to_db)
conn.commit()
f.close()

# Exploring Data

## Size of CSV Files

In [33]:
#links to the csv files
nodes_csv = 'nodes.csv'
ways_csv = 'ways.csv'
nodestags_csv = 'nodes_tags.csv'
waystags_csv = 'ways_tags.csv'
waysnodes_csv = 'ways_nodes.csv'

# Get the size (in bytes) of specified path  
size_tricitiesdb = os.path.getsize('tricities.db')
size_osm_xml = os.path.getsize('map.osm')
size_nodes = os.path.getsize(nodes_csv) 
size_ways = os.path.getsize(ways_csv) 
size_nodestags = os.path.getsize(nodestags_csv) 
size_waystags = os.path.getsize(waystags_csv) 
size_waysnodes = os.path.getsize(waysnodes_csv) 
  
# Print the size (in bytes) of specified path  
print("Size (In bytes) of '%s':" %'tri_cities.db', size_tricitiesdb)
print("Size (In bytes) of '%s':" %'map.osm', size_osm_xml)
print("Size (In bytes) of '%s':" %nodes_csv, size_nodes)
print("Size (In bytes) of '%s':" %ways_csv, size_ways)
print("Size (In bytes) of '%s':" %nodestags_csv, size_nodestags)
print("Size (In bytes) of '%s':" %waystags_csv, size_waystags)
print("Size (In bytes) of '%s':" %waysnodes_csv, size_waysnodes)

("Size (In bytes) of 'tri_cities.db':", 42356736L)
("Size (In bytes) of 'map.osm':", 82031159L)
("Size (In bytes) of 'nodes.csv':", 31701618L)
("Size (In bytes) of 'ways.csv':", 2396620L)
("Size (In bytes) of 'nodes_tags.csv':", 1158419L)
("Size (In bytes) of 'ways_tags.csv':", 4416724L)
("Size (In bytes) of 'ways_nodes.csv':", 10807215L)


## Number of Nodes

In [34]:
query = "SELECT count(DISTINCT(id)) FROM nodes;"
cur.execute(query)
rows=cur.fetchall()

pprint.pprint(rows)

[(368263,)]


## Number of Ways

In [35]:
query = "SELECT count(DISTINCT(id)) FROM ways;"
cur.execute(query)
rows=cur.fetchall()

pprint.pprint(rows)

[(38417,)]


## Top Contributing Users

In [36]:
query= 'SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 25;'
cur.execute(query)
rows = cur.fetchall()
pprint.pprint(rows)

[(u'Jessica12345', 102374),
 (u'Howpper', 68010),
 (u'DJ Cane', 38615),
 (u'OrcaDan', 23035),
 (u'woodpeck_fixbot', 15212),
 (u'Sundance', 14827),
 (u'miroslavuzice87', 11778),
 (u'bab72', 9732),
 (u'Natfoot', 9487),
 (u'Brad Meteor', 8091),
 (u'shutle64', 6252),
 (u'zephyr', 4888),
 (u'Heptazane', 4599),
 (u'caseyb', 4553),
 (u'Glassman', 4247),
 (u'cowdog', 4133),
 (u'SuperFlomm', 3086),
 (u'Something B', 2881),
 (u'Dilys', 2311),
 (u'Aaron Lidman', 2199),
 (u'TheDutchMan13', 2105),
 (u'Timothy Whidden', 2103),
 (u'eyewitness', 2097),
 (u'leuty', 2034),
 (u'PHerison', 1980)]


## Type and Number of Nodes

In [37]:
query = "SELECT type , count(*) as num  FROM nodes_tags group by type order by num desc;"
cur.execute(query)
rows=cur.fetchall()

pprint.pprint(rows)

[(u'regular', 28310),
 (u'addr', 854),
 (u'gnis', 417),
 (u'generator', 341),
 (u'brand', 333),
 (u'railway', 120),
 (u'tower', 40),
 (u'seamark', 39),
 (u'traffic_signals', 23),
 (u'name', 22),
 (u'fire_hydrant', 21),
 (u'toilets', 19),
 (u'opening_hours', 15),
 (u'healthcare', 13),
 (u'socket', 11),
 (u'flag', 9),
 (u'monitoring', 6),
 (u'authentication', 6),
 (u'was', 4),
 (u'census', 4),
 (u'handwashing', 3),
 (u'disused', 3),
 (u'turn', 2),
 (u'ref', 2),
 (u'parking', 2),
 (u'operator', 2),
 (u'is_in', 2),
 (u'internet_access', 2),
 (u'crossing', 2),
 (u'contact', 2),
 (u'charging_station', 2),
 (u'wetap', 1),
 (u'survey', 1),
 (u'stop', 1),
 (u'roof', 1),
 (u'payment', 1),
 (u'fuel', 1),
 (u'diet', 1),
 (u'building', 1),
 (u'aerodrome', 1),
 (u'abandoned', 1)]


## Type and Number of Ways

In [38]:
query = "SELECT type , count(*) as num  FROM ways_tags group by type order by num desc;"
cur.execute(query)
rows=cur.fetchall()

pprint.pprint(rows)

[(u'regular', 79771),
 (u'tiger', 37677),
 (u'addr', 2502),
 (u'source', 437),
 (u'building', 405),
 (u'brand', 303),
 (u'roof', 296),
 (u'lanes', 210),
 (u'gnis', 206),
 (u'turn', 180),
 (u'hgv', 117),
 (u'destination', 95),
 (u'name', 78),
 (u'maxspeed', 63),
 (u'parking', 43),
 (u'cycleway', 41),
 (u'payment', 39),
 (u'opening_hours', 21),
 (u'crossing', 15),
 (u'abandoned', 14),
 (u'disused', 13),
 (u'is_in', 11),
 (u'operator', 10),
 (u'internet_access', 8),
 (u'fuel', 8),
 (u'healthcare', 7),
 (u'ref', 6),
 (u'historic', 5),
 (u'bridge', 5),
 (u'surface', 4),
 (u'seamark', 3),
 (u'razed', 2),
 (u'note', 2),
 (u'heritage', 2),
 (u'footway', 2),
 (u'diet', 2),
 (u'contact', 2),
 (u'wikipedia', 1),
 (u'toilets', 1),
 (u'theatre', 1),
 (u'socket', 1),
 (u'service_times', 1),
 (u'planned', 1),
 (u'observatory', 1),
 (u'capacity', 1)]


## Biggest Religions

In [39]:
query="select value, count(*) as num from (select key,value from nodes_tags UNION ALL select key,value from ways_tags) as e  where key='religion' group by value order by num desc limit 10;"
cur.execute(query)
rows=cur.fetchall()

pprint.pprint(rows)

[(u'christian', 37), (u'muslim', 1), (u'hindu', 1)]


## Top Quisines

In [40]:
query="select value,count(*) as num from (select key,value from nodes_tags UNION ALL select key,value from ways_tags) as e where e.key like '%cuisine%' group by value order by num desc limit 25;"
cur.execute(query)
rows=cur.fetchall()

pprint.pprint(rows)

[(u'burger', 36),
 (u'coffee_shop', 23),
 (u'pizza', 19),
 (u'mexican', 18),
 (u'sandwich', 13),
 (u'american', 9),
 (u'tex-mex', 7),
 (u'chinese', 6),
 (u'seafood', 4),
 (u'ice_cream;burger', 4),
 (u'chicken', 4),
 (u'asian', 4),
 (u'thai', 3),
 (u'italian', 3),
 (u'wings', 2),
 (u'juice', 2),
 (u'breakfast;pancake', 2),
 (u'sushi', 1),
 (u'steak_house;sushi', 1),
 (u'steak_house', 1),
 (u'pretzel', 1),
 (u'mongolian', 1),
 (u'mediterranean', 1),
 (u'korean', 1),
 (u'kebab', 1)]


## Top Amenities

In [41]:
query="select value, count(*) as num from nodes_tags where key='amenity' group by value order by num desc limit 25;"
cur.execute(query)
rows=cur.fetchall()

pprint.pprint(rows)

[(u'bench', 151),
 (u'restaurant', 52),
 (u'toilets', 51),
 (u'fast_food', 46),
 (u'parking', 30),
 (u'waste_basket', 29),
 (u'shelter', 28),
 (u'school', 22),
 (u'vending_machine', 21),
 (u'fuel', 20),
 (u'bicycle_parking', 16),
 (u'cafe', 15),
 (u'place_of_worship', 12),
 (u'drinking_water', 11),
 (u'bank', 11),
 (u'dentist', 10),
 (u'atm', 9),
 (u'clinic', 8),
 (u'post_box', 6),
 (u'pharmacy', 6),
 (u'loading_dock', 6),
 (u'ice_cream', 4),
 (u'fountain', 4),
 (u'post_office', 3),
 (u'letter_box', 3)]


## Number of Unique Users

In [42]:
query = "SELECT COUNT(DISTINCT(e.uid))FROM (SELECT uid FROM Nodes UNION ALL SELECT uid FROM Ways) as e;"
cur.execute(query)
rows=cur.fetchall()

pprint.pprint(rows)

[(447,)]


## Additional Improvement

> Additional improvement that could be made to the dataset is just overall human error correction. Any dataset this large that is entirely comprised of human input will have errors and doing cleaning and analysis like this on all types of different data points would make the data much more uniform across the dataset. This could be improved by having a uniform data entering scheme that all users of OpenStreetMaps have to follow to submit data. This could help keep the data between different users much more uniform in general.

# Conclusion

> This dataset is very large and comprised entirely of information provided by many different users. Different users entering different data at different times leads to some messy data. Going through and cleaning the data like this helps to make the data overall much more uniform and readable. Having a standardized data entering scheme would keep the data much more uniform from the start, however, for a completely open-source website that anyone can enter on, the data was overall very clean. It certainly could have been much worse. The way users enter data is probably about as good as it can be, and for users who prefer more clean data, and analysis such as this could be carried out to standardize all the data.